In [27]:
%load_ext autoreload
%autoreload 2

In [28]:
import numpy as np
import pandas as pd
from utils import run_ML

In [29]:
# !pip install tldextract

In [30]:
data_dir = "data/URLdatasetX2_1.csv"
df = pd.read_csv(data_dir,index_col=0)

In [31]:
df.shape, df.head(2)

((2802, 2),
                                           url        type
 0       http://www.crestonwood.com/router.php  legitimate
 1  http://vamoaestudiarmedicina.blogspot.com/  legitimate)

In [32]:
# smalldata = df.sample(n = 20000, random_state=1)
smalldata = df

In [33]:
if 'PhiUSIIL' in data_dir:
    urldata = np.array(smalldata[['URLLength', 'DomainLength', 'IsDomainIP', \
           'URLSimilarityIndex', 'CharContinuationRate', 'TLDLegitimateProb',\
           'URLCharProb', 'TLDLength', 'NoOfSubDomain', 'HasObfuscation',\
           'NoOfObfuscatedChar', 'ObfuscationRatio', 'NoOfLettersInURL',\
           'LetterRatioInURL', 'NoOfDegitsInURL', 'DegitRatioInURL',\
           'NoOfEqualsInURL', 'NoOfQMarkInURL', 'NoOfAmpersandInURL',\
           'NoOfOtherSpecialCharsInURL', 'SpacialCharRatioInURL', 'IsHTTPS']])
    labels = smalldata.iloc[:,-1].values
    run_ML(urldata, labels, "PhiUSIILmock", "standard")
if 'URLdatasetX2' in data_dir:
    labels = smalldata.iloc[:,-1].values
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

In [34]:
# labels

### Feature extraction

In [35]:
from utils import extract_features

In [36]:
# Example usage:
url = "http://www.example.com/path/to/==file.html"
url_features = extract_features(url)
print(url_features)

{'domain': 'www.example.com', 'num_subdomains': 2, 'contains_ip': 0, 'path_length': 20, 'num_path_segments': 3, 'uses_https': 0, 'file_extension': 'html', 'count_special_characters': 11, 'count_non_alphanumeric_characters': 11, 'TLD': 'com', 'count_obfuscated_characters': 0, 'letter_ratio_in_url': 0.7380952380952381, 'digit_ratio_in_url': 0.0, 'count_equals_in_url': 2, 'NoOfAmpersandInURL': 0, 'CharContinuationRate': 0.11904761904761904, 'ratio_obfuscated_characters': 0.0, 'NoOfQMarkInURL': 0}


In [37]:
# print(url_features.keys())

In [38]:
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_features(link)
    phish_url.append(list(url_features.values())[1:])

In [39]:
phish_url_df = pd.DataFrame(phish_url, columns = list(url_features.keys())[1:])

In [40]:
# phish_url_df.head(2)

In [41]:
phish_url_df.iloc[:,5] = pd.Categorical(phish_url_df.iloc[:,5]).codes
phish_url_df.iloc[:,8] = pd.Categorical(phish_url_df.iloc[:,8]).codes

In [42]:
phish_url_df.head(2)

,num_subdomains,contains_ip,path_length,num_path_segments,uses_https,file_extension,count_special_characters,count_non_alphanumeric_characters,TLD,count_obfuscated_characters,letter_ratio_in_url,digit_ratio_in_url,count_equals_in_url,NoOfAmpersandInURL,CharContinuationRate,ratio_obfuscated_characters,NoOfQMarkInURL
0,2,0,11,1,0,47,7,7,38,0,0.810811,0.0,0,0,0.135135,0.0,0
1,2,0,1,1,0,0,6,6,38,0,0.857143,0.0,0,0,0.047619,0.0,0


In [43]:
run_ML(phish_url_df, labels, "URLdatasetX2", "manual")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.86]


In [47]:
## Numerical features
from utils import extract_numerical_features
phish_url = []
for link in list(smalldata.iloc[:,0]):
    url_features = extract_numerical_features(link)
    phish_url.append(list(url_features.values()))
run_ML(np.array(phish_url), labels, "URLdatasetX2", "manual_numerical")

Run:  0 , fold:  0
Train freq:  [1630, 611]
kNN, LightGBM, Run:  0 , fold:  1
Train freq:  [1617, 624]
kNN, LightGBM, Run:  0 , fold:  2
Train freq:  [1602, 640]
kNN, LightGBM, Run:  0 , fold:  3
Train freq:  [1645, 597]
kNN, LightGBM, Run:  0 , fold:  4
Train freq:  [1622, 620]
kNN, LightGBM, ['kNN', 'LightGBM']
[0.73 0.83]


In [76]:
import requests
from bs4 import BeautifulSoup

In [80]:
hyperlink_features = np.zeros((smalldata.shape[0], 48))
enum = 0;
for url in list(smalldata.iloc[:,0]):
    # url = 'https://polarklimatsgserver.blogspot.com/'
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    urls = []
    count = 0;
    for link in soup.find_all('a'):
        # print(link.get('href'))
        weblink = link.get('href')
        if (weblink is not None) and ('http' in weblink):
            urls.append(weblink)
        count += 1
        if count > 50:
            break
    # extract numerical features in link
    if len(urls) > 0:
        hyperlink_data = []
        for link in urls:
            url_features = extract_numerical_features(link)
            hyperlink_data.append(list(url_features.values()))
        hyperlink_data = np.array(hyperlink_data)
        hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),
                              hyperlink_data.min(axis=0),hyperlink_data.max(axis=0), hyperlink_data.mean(axis=0)))
    else:
        hyper_np = np.hstack((np.array([len(urls), count, float(len(urls))/(count + 1)]),np.zeros(45)))
    hyperlink_features[enum, :] = hyper_np
    print(enum, end =',')
    enum += 1

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021ABA2C1DC0>>
Traceback (most recent call last):
  File "C:\Users\local-admin\.conda\envs\py39\lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt

